# Problem 

Optimal passengers allocation to transport helicopters, maximising vehicle efficiency whilst being robust against uncertainty  

Variables and conditions:  
- 10 assets requiring maintainance  
- 4 helicopters available of two capacities (15 and 25 passengers)  
- normally distributed staff, with mean 60 and standard deviation 20  
- passengers groupped in unsplittable teams of 1 to 8 members  
- helicopters can fly to more than one location  
- assets' staff demand varies uniformly every day  
- multiple teams allocatable to each location  
- no individual team allocatable to multiple locations  
- negligible flight time   

Additional aspects to consider:  
- different importance/urgency of teams' task at each asset  
- the presence of uncertainties  

# Solution

## Methodic

The business question presents similarities with the _Knapsack problem_ of efficient allocation  
of differently valued and sized items within fixed capacity containers. Given the additional  
levels of complexity listed above, a solution can be obtained implementing an expanded and  
"recursive" version of the [Knapsack Problem](https://en.wikipedia.org/wiki/Knapsack_problem). 
Below I summarise the various steps of this approach  
1. **setting staff availability and assets' demand**: for a given day, characterise each asset's demand(s) and the staff availability.  
Match the assets' demand(s) with the available staff to get the allocatable teams.  
2. **recursive helicopter assignment**: construct and retain all possible teams allocation patterns.  
Recursive application of the _Knapsack problem_ solution recipe until all aircrafts are occupied and/or the teams allocated.  
Repetition of the procedure for different vehicle sequences, to find the teams allocation patterns yielding the highest value.  
3. **optimal pattern selection**: apply conditions on assets' demands and teams' task to determine most efficient pattern.  
Search for optimal pattern applying efficiency criteria (eg. largest number of teams assigned to smallest number of helicopters)

## Pragmatic

Exploring all possible allocation patterns by *brute force* approach: simulate various scenarios.  
This approach can be summarised in the following steps.  
1. **setting staff availability and assets' demand**: characterise daily asset's demand and staff availability.  
2. **random teams allocation patterns generation**: simulate possible teams assignment to helicopters.  
3. **highest scoring pattern selection**: retain the random pattern achieving the highest score.  

#### uncertainties
flight uncertainty = good weather probability at asset's location $\times$ helicopter's functionality

# Implementation: Methodic

I present a numerical implementation of the methodic solution introduced above.

## 1) Setting staff availability and assets' demand

In [23]:
import numpy as np

from Logistics import *

ndays = 7    # number of days simulated
ntasks = 7   # maximum number of daily tasks
nvalues = 5  # number of task value classes
nassets = 10 # number of assets
teamsize = 8 # max teams size

Helicopters = [15,25]*2   # helicopters capacity

### assets' demands

For every asset  
- assign a _good weather_ probability
- generate a number of requested tasks  
- assign to each task an importance value   
- assign to each task a requested staff number  

Below an example of a possible set of daily demands:

In [24]:
asset_demand(nassets, ntasks, nvalues, teamsize)

[{'a': 1, 'k': 1, 'v': 1, 's': 6, 'w': 0.2644572275157878},
 {'a': 1, 'k': 2, 'v': 5, 's': 8, 'w': 0.2644572275157878},
 {'a': 1, 'k': 3, 'v': 1, 's': 2, 'w': 0.2644572275157878},
 {'a': 1, 'k': 4, 'v': 1, 's': 4, 'w': 0.2644572275157878},
 {'a': 1, 'k': 5, 'v': 2, 's': 8, 'w': 0.2644572275157878},
 {'a': 2, 'k': 1, 'v': 4, 's': 7, 'w': 0.015762026848746058},
 {'a': 2, 'k': 2, 'v': 2, 's': 4, 'w': 0.015762026848746058},
 {'a': 2, 'k': 3, 'v': 2, 's': 8, 'w': 0.015762026848746058},
 {'a': 2, 'k': 4, 'v': 1, 's': 6, 'w': 0.015762026848746058},
 {'a': 2, 'k': 5, 'v': 5, 's': 1, 'w': 0.015762026848746058},
 {'a': 3, 'k': 1, 'v': 3, 's': 7, 'w': 0.2020319640862619},
 {'a': 3, 'k': 2, 'v': 2, 's': 8, 'w': 0.2020319640862619},
 {'a': 4, 'k': 1, 'v': 3, 's': 4, 'w': 0.32242385339696855},
 {'a': 4, 'k': 2, 'v': 2, 's': 4, 'w': 0.32242385339696855},
 {'a': 4, 'k': 3, 'v': 4, 's': 6, 'w': 0.32242385339696855},
 {'a': 4, 'k': 4, 'v': 3, 's': 5, 'w': 0.32242385339696855},
 {'a': 4, 'k': 5, 'v': 4, 

where:  
- 'a' asset label  
- 'k' task label  
- 'v' task value label  
- 's' team size requested  
- 'w' good weather probability

### staff availability

Daily available staff: a (normally distributed) number, randomly split into different task value classes

Below an example of a possible set of daily staff availability per task:

In [25]:
staff_available(nvalues)

[{'v': 1, 's': 1},
 {'v': 2, 's': 27},
 {'v': 3, 's': 21},
 {'v': 4, 's': 7},
 {'v': 5, 's': 6}]

### teams formation

Matching an asset's demand(s) with the staff available.  

Below an example of the teams-matching outcome on a certain day

In [26]:
demand = asset_demand(nassets, ntasks, nvalues, teamsize)
staff = staff_available(nvalues)

teams, left = demand_staff_assign(demand, staff, nvalues)
for t in teams:
    print(t)

{'T': 'T_1', 'A': 'A_9', 'V': 1, 'S': 7, 'W': 0.6701904737985434, 'K': 'K_9_1'}
{'T': 'T_2', 'A': 'A_8', 'V': 1, 'S': 6, 'W': 0.282567000792029, 'K': 'K_8_3'}
{'T': 'T_3', 'A': 'A_4', 'V': 1, 'S': 5, 'W': 0.8225984980156964, 'K': 'K_4_4'}
{'T': 'T_4', 'A': 'A_3', 'V': 1, 'S': 4, 'W': 0.26537336028453073, 'K': 'K_3_3'}
{'T': 'T_5', 'A': 'A_5', 'V': 1, 'S': 3, 'W': 0.9560473747060789, 'K': 'K_5_1'}
{'T': 'T_6', 'A': 'A_4', 'V': 3, 'S': 3, 'W': 0.8225984980156964, 'K': 'K_4_1'}
{'T': 'T_7', 'A': 'A_8', 'V': 3, 'S': 2, 'W': 0.282567000792029, 'K': 'K_8_4'}
{'T': 'T_8', 'A': 'A_3', 'V': 4, 'S': 8, 'W': 0.26537336028453073, 'K': 'K_3_1'}
{'T': 'T_9', 'A': 'A_4', 'V': 4, 'S': 8, 'W': 0.8225984980156964, 'K': 'K_4_2'}
{'T': 'T_10', 'A': 'A_2', 'V': 4, 'S': 3, 'W': 0.8077853516301693, 'K': 'K_2_2'}
{'T': 'T_11', 'A': 'A_2', 'V': 5, 'S': 3, 'W': 0.8077853516301693, 'K': 'K_2_1'}


## 2) Recursive helicopter assignment

Teams allocation onto the helicopters. 

Below an example of the "recursive" application of the _Knapsack problem_ solution recipe on the available helicopters

In [28]:
# simulate daily helicopters functionality 
Hfunc = np.random.rand(len(Helicopters))
# simulate daily assets' demand(s)
demand = asset_demand(nassets, ntasks, nvalues, teamsize)
# simulate daily available staff
staff = staff_available(nvalues)
# match demands with staff to create teams
Teams, left = demand_staff_assign(demand, staff, nvalues)
# build possible allocation patterns
Patterns = daily_teams_allocation(Helicopters, Hfunc, Teams)

import texttable
tab = texttable.Texttable()

headings = ['pattern', 'N teams', 'N helicopters', 'teams (labels)', 'assets (labels)']
tab.header(headings)

NN = Patterns['N']
AA = Patterns['A']
TT = Patterns['T']

for p in range(len(AA)):
    N = NN[p]
    h = [i for (i,a) in enumerate(AA[p]) if a]
    A = [AA[p][j] for j in h]
    T = [TT[p][j] for j in h]
    tab.add_row([p+1, N, len(h), T, A])

tab.set_cols_align(['c','c','c','l','l'])
tab.set_cols_width([8,8,15,60,60])
s = tab.draw()
print(s)

+----------+----------+-----------------+--------------------------------------------------------------+--------------------------------------------------------------+
| pattern  | N teams  |  N helicopters  |                        teams (labels)                        |                       assets (labels)                        |
+==========+==========+=================+==============================================================+==============================================================+
|    1     |    8     |        3        | [['T_8', 'T_6', 'T_5'], ['T_7', 'T_4', 'T_3', 'T_2'],        | [['A_7', 'A_2', 'A_7'], ['A_8', 'A_4', 'A_8', 'A_8'],        |
|          |          |                 | ['T_1']]                                                     | ['A_9']]                                                     |
+----------+----------+-----------------+--------------------------------------------------------------+--------------------------------------------------------

## 3) optimal pattern selection

Systematically determine the optimal pattern among the possible ones by progressively applying conditions on their features.  

#### maximum value patterns

Determine the highest value patterns and neglect the others

In [29]:
maxV_patterns, SV = max_value_patterns(Patterns)

#### least helicopters used patterns

Determine the patterns requiring the smallest number of helicopters and neglect the others

In [30]:
minH_patterns, SH = least_helicopters_patterns(maxV_patterns)

#### most teams allocated patterns

Determine the patterns yielding the largest number of teams being allocated and neglect the others  
(the action of this function is generally equivalent to that of `max_value_patterns`)

In [31]:
maxN_patterns, SN = most_teams_patterns(minH_patterns)

#### similar destinations patterns

Determine the patterns scoring highest for assets' location homogeneity and neglect the others.  
(eg. score of [3, 3, 3, 7, 7, 5] > score of [3, 4, 5, 6, 7, 8] )

In [32]:
maxA_patterns, SA = similar_destinations_patterns(maxN_patterns)

#### max probability patterns

Determine the patterns scoring highest in flying probability and neglect the others.  
(eg. score of [78%, 82%, 95%] > score of [48%, 52%, 75%] )

In [33]:
maxP_patterns, SP = max_probability_patterns(maxA_patterns)

Below the optimal pattern resulting from the determination process

In [34]:
tab = texttable.Texttable()

headings = ['N teams', 'N helicopters', 'teams (labels)', 'assets (labels)']
tab.header(headings)

NN = maxP_patterns['N'][0]
AA = maxP_patterns['A'][0]
TT = maxP_patterns['T'][0]

h = [i for (i,a) in enumerate(AA) if a]
H = len([Helicopters[j] for j in h])
A = [AA[j] for j in h]
T = [TT[j] for j in h]
    
tab.add_row([NN, H, T, A])

tab.set_cols_align(['c','c','l','l'])
tab.set_cols_width([10,15,60,60])
s = tab.draw()
print(s)

+------------+-----------------+--------------------------------------------------------------+--------------------------------------------------------------+
|  N teams   |  N helicopters  |                        teams (labels)                        |                       assets (labels)                        |
+============+=================+==============================================================+==============================================================+
|     8      |        2        | [['T_8', 'T_7', 'T_6', 'T_5'], ['T_4', 'T_3', 'T_2', 'T_1']] | [['A_7', 'A_8', 'A_2', 'A_7'], ['A_4', 'A_8', 'A_8', 'A_9']] |
+------------+-----------------+--------------------------------------------------------------+--------------------------------------------------------------+


### repeated application

We can put all together and observe how this whole setup works for an entire week of operation

In [35]:
tab = texttable.Texttable()

headings = ['day', 'staff', 'Score', 'Teams (labels)', 'Assets (labels)']
tab.header(headings)

# simulate daily helicopters functionality 
Hfunc = np.random.rand(len(Helicopters))

left = None
for d in range(ndays):
    demand = asset_demand(nassets, ntasks, nvalues, teamsize)
    staff = staff_available(nvalues)
    S = sum([st['s'] for st in staff])
    teams, left = demand_staff_assign(demand, staff, nvalues, old=left)
    Patterns = daily_teams_allocation(Helicopters, Hfunc, teams)
    maxV_patterns, SV = max_value_patterns(Patterns)
    minH_patterns, SH = least_helicopters_patterns(maxV_patterns)
    maxN_patterns, SN = most_teams_patterns(minH_patterns)
    maxA_patterns, SA = similar_destinations_patterns(maxN_patterns)
    maxP_patterns, SP = max_probability_patterns(maxA_patterns)

    A = maxP_patterns['A'][0]
    T = maxP_patterns['T'][0]

    h = [i for (i,a) in enumerate(A) if a]
    A = [A[j] for j in h]
    T = [T[j] for j in h]
    
    score = SV * SH * SN * SA * SP
    tab.add_row([d+1, S, score, T, A])

tab.set_cols_align(['c','c','c','l','l'])
tab.set_cols_dtype(['i','i','e','a','a'])
tab.set_cols_width([3,5,7,70,70])
s = tab.draw()
print (s)

+-----+-------+---------+------------------------------------------------------------------------+------------------------------------------------------------------------+
| day | staff |  Score  |                             Teams (labels)                             |                            Assets (labels)                             |
+=====+=======+=========+========================================================================+========================================================================+
|  1  |  91   |  0.033  | [['T_15', 'T_14', 'T_13', 'T_10', 'T_4', 'T_3', 'T_2'], ['T_12',       | [['A_2', 'A_9', 'A_7', 'A_7', 'A_4', 'A_4', 'A_1'], ['A_4', 'A_9'],    |
|     |       |         | 'T_9'], ['T_11', 'T_8'], ['T_7', 'T_6', 'T_5']]                        | ['A_3', 'A_2'], ['A_5', 'A_4', 'A_3']]                                 |
+-----+-------+---------+------------------------------------------------------------------------+------------------------------------------

# Implementation: Pragmatic

## 1) Setting staff availability and assets' demand

This is equivalent to first step in the _Methodic_ solution presented above.  

In [36]:
# simulate daily helicopters functionality 
Hfunc = np.random.rand(len(Helicopters))
# simulate daily assets' demand(s)
demand = asset_demand(nassets, ntasks, nvalues, teamsize)
# simulate daily available staff
staff = staff_available(nvalues)
# match demands with staff to create teams
Teams, left = demand_staff_assign(demand, staff, nvalues)

## 2) random teams allocation patterns generation

Teams allocation onto the helicopters.

Below an example of the _brute-force_ random assignment to the available helicopters.

In [39]:
tab = texttable.Texttable()

headings = ['i', 'Score', 'teams (labels)', 'assets (labels)']
tab.header(headings)

Sarr, Harr, Tarr, Aarr = random_teams_allocator(Helicopters, Hfunc, Teams)

for it in range(len(Sarr)):
    score = Sarr[it]
    TT = Tarr[it]
    AA = Aarr[it]
    
    h = [i for (i,a) in enumerate(AA) if a]
    A = [AA[j] for j in h]
    T = [TT[j] for j in h]
    
    tab.add_row([it+1, score, T, A])

tab.set_cols_align(['c','c','l','l'])
tab.set_cols_dtype(['i','e','a','a'])
tab.set_cols_width([2,5,70,70])
s = tab.draw()
print(s)

+----+-------+------------------------------------------------------------------------+------------------------------------------------------------------------+
| i  | Score |                             teams (labels)                             |                            assets (labels)                             |
+====+=======+========================================================================+========================================================================+
| 1  | 0.024 | [['T_1', 'T_2', 'T_3'], ['T_4', 'T_5', 'T_6', 'T_7', 'T_8', 'T_9',     | [['A_9', 'A_1', 'A_2'], ['A_6', 'A_7', 'A_6', 'A_4', 'A_7', 'A_6',     |
|    |       | 'T_10']]                                                               | 'A_1']]                                                                |
+----+-------+------------------------------------------------------------------------+------------------------------------------------------------------------+
| 2  | 0.496 | [['T_1', 'T_5', 'T_

## 3) highest scoring pattern selection 

Retain the higest scoring pattern among the ones obtained.

In [40]:
ind_max = np.where(Sarr==max(Sarr))[0][0]

tab = texttable.Texttable()

headings = ['i', 'Score', 'teams (labels)', 'assets (labels)']
tab.header(headings)

Sbest = Sarr[ind_max] 
Hbest = Harr[ind_max] 
Tbest = Tarr[ind_max]
Abest = Aarr[ind_max]

h = [i for (i,a) in enumerate(Abest) if a]
A = [Abest[j] for j in h]
T = [Tbest[j] for j in h]

tab.add_row([ind_max+1, Sbest, T, A])

tab.set_cols_align(['c','c','l','l'])
tab.set_cols_width([2,5,70,70])
s = tab.draw()
print(s)

+----+-------+------------------------------------------------------------------------+------------------------------------------------------------------------+
| i  | Score |                             teams (labels)                             |                            assets (labels)                             |
+====+=======+========================================================================+========================================================================+
| 7  | 1.153 | [['T_2', 'T_8', 'T_10', 'T_5', 'T_9', 'T_4', 'T_6', 'T_3'], ['T_7',    | [['A_1', 'A_7', 'A_1', 'A_7', 'A_6', 'A_6', 'A_6', 'A_2'], ['A_4',     |
|    |       | 'T_1']]                                                                | 'A_9']]                                                                |
+----+-------+------------------------------------------------------------------------+------------------------------------------------------------------------+


# Reliability & Efficiency

## Efficiency
Compare the efficiency of the _Methodic_ and Pragmatic_ approaches.  

First, let's derive a reference pattern using the methodic approach introduced

In [41]:
# simulate daily assets' demand(s)
demand = asset_demand(nassets, ntasks, nvalues, teamsize)
# simulate daily available staff
staff = staff_available(nvalues)
# match demands with staff to create teams
Teams, left = demand_staff_assign(demand, staff, nvalues)
# simulate daily helicopters functionality
Hfunc = np.random.rand(len(Helicopters))
# build possible allocation patterns
Patterns = daily_teams_allocation(Helicopters, Hfunc, Teams)
# get maximum value patterns and corresponding score
maxV_patterns, SV = max_value_patterns(Patterns)
# get smallest number of used helicopters patterns and corresponding score
minH_patterns, SH = least_helicopters_patterns(maxV_patterns)
# get largest allocated teams patterns and corresponding score
maxN_patterns, SN = most_teams_patterns(minH_patterns)
# get most destination efficient patterns and corresponding score
maxA_patterns, SA = similar_destinations_patterns(maxN_patterns)
# get highest chances pattern and corresponding score
maxP_patterns, SP = max_probability_patterns(maxA_patterns)

A = maxP_patterns['A'][0]
T = maxP_patterns['T'][0]

h = [i for (i,a) in enumerate(A) if a]
H = [Helicopters[j] for j in h]
A = [A[j] for j in h]
T = [T[j] for j in h]

score = SV * SH * SN * SA * SP
score_ref, T_ref, A_ref = score, T, A

Now compare the performance of the _Methodic_ solution with the _Pragmatic_ (random) one

In [42]:
tab1 = texttable.Texttable()

headings = ['i', 'Score', 'teams (labels)', 'assets (labels)']
tab1.header(headings)

tab1.add_row(['Ref', score_ref, T_ref, A_ref])

tab1.set_cols_align(['c','c','l','l'])
tab1.set_cols_width([3,6,70,70])
s = tab1.draw()
print('Reference (Methodic)')
print(s)
print()

tab2 = texttable.Texttable()

headings = ['i', 'Score', 'teams (labels)', 'assets (labels)']
tab2.header(headings)

Sarr, Harr, Tarr, Aarr = random_teams_allocator(Helicopters, Hfunc, Teams)

for it in range(len(Sarr)):
    score = Sarr[it]
    TT = Tarr[it]
    AA = Aarr[it]
    
    h = [i for (i,a) in enumerate(AA) if a]
    A = [AA[j] for j in h]
    T = [TT[j] for j in h]
    
    tab2.add_row([it+1, score, T, A])

tab2.set_cols_align(['c','c','l','l'])
tab2.set_cols_width([3,6,70,70])
s = tab2.draw()
print('Random iterations')
print(s)

Reference (Methodic)
+-----+--------+------------------------------------------------------------------------+------------------------------------------------------------------------+
|  i  | Score  |                             teams (labels)                             |                            assets (labels)                             |
+=====+========+========================================================================+========================================================================+
| Ref | 13.189 | [['T_5', 'T_4', 'T_3', 'T_2'], ['T_1']]                                | [['A_4', 'A_2', 'A_2', 'A_6'], ['A_6']]                                |
+-----+--------+------------------------------------------------------------------------+------------------------------------------------------------------------+

Random iterations
+-----+--------+------------------------------------------------------------------------+--------------------------------------------------------

Increase number of aircrafts to better visualise differences in efficiency

In [43]:
Helicopters = Helicopters*2

In [44]:
# simulate daily assets' demand(s)
demand = asset_demand(nassets, ntasks, nvalues, teamsize)
# simulate daily available staff
staff = staff_available(nvalues)
# match demands with staff to create teams
Teams, left = demand_staff_assign(demand, staff, nvalues)
# simulate daily helicopters functionality
Hfunc = np.random.rand(len(Helicopters))
# build possible allocation patterns
Patterns = daily_teams_allocation(Helicopters, Hfunc, Teams)
# get maximum value patterns and corresponding score
maxV_patterns, SV = max_value_patterns(Patterns)
# get smallest number of used helicopters patterns and corresponding score
minH_patterns, SH = least_helicopters_patterns(maxV_patterns)
# get largest allocated teams patterns and corresponding score
maxN_patterns, SN = most_teams_patterns(minH_patterns)
# get most destination efficient patterns and corresponding score
maxA_patterns, SA = similar_destinations_patterns(maxN_patterns)
# get highest chances pattern and corresponding score
maxP_patterns, SP = max_probability_patterns(maxA_patterns)

A = maxP_patterns['A'][0]
T = maxP_patterns['T'][0]

h = [i for (i,a) in enumerate(A) if a]
H = [Helicopters[j] for j in h]
A = [A[j] for j in h]
T = [T[j] for j in h]

score = SV * SH * SN * SA * SP
score_ref, T_ref, A_ref = score, T, A

Now compare the performance of the _Methodic_ solution with the _Pragmatic_ (random) one

In [45]:
tab1 = texttable.Texttable()

headings = ['i', 'Score', 'teams (labels)', 'assets (labels)']
tab1.header(headings)

tab1.add_row(['Ref', score_ref, T_ref, A_ref])

tab1.set_cols_align(['c','c','l','l'])
tab1.set_cols_width([3,6,70,70])
s = tab1.draw()
print('Reference (Methodic)')
print(s)
print()

tab2 = texttable.Texttable()

headings = ['i', 'Score', 'teams (labels)', 'assets (labels)']
tab2.header(headings)

Sarr, Harr, Tarr, Aarr = random_teams_allocator(Helicopters, Hfunc, Teams)

for it in range(len(Sarr)):
    score = Sarr[it]
    TT = Tarr[it]
    AA = Aarr[it]
    
    h = [i for (i,a) in enumerate(AA) if a]
    A = [AA[j] for j in h]
    T = [TT[j] for j in h]
    
    tab2.add_row([it+1, score, T, A])

tab2.set_cols_align(['c','c','l','l'])
tab2.set_cols_width([3,6,70,70])
s = tab2.draw()
print('Random iterations')
print(s)

Reference (Methodic)
+-----+--------+------------------------------------------------------------------------+------------------------------------------------------------------------+
|  i  | Score  |                             teams (labels)                             |                            assets (labels)                             |
+=====+========+========================================================================+========================================================================+
| Ref | 0.004  | [['T_14', 'T_13', 'T_12', 'T_11', 'T_10', 'T_6'], ['T_9', 'T_8',       | [['A_6', 'A_4', 'A_1', 'A_6', 'A_4', 'A_7'], ['A_3', 'A_10', 'A_4',    |
|     |        | 'T_7', 'T_5', 'T_4', 'T_3'], ['T_2', 'T_1']]                           | 'A_7', 'A_6', 'A_7'], ['A_1', 'A_8']]                                  |
+-----+--------+------------------------------------------------------------------------+------------------------------------------------------------------------+



## Reliability 

### Differing values choices 

Assess robustness of the method for different choices of `ntasks` and `nvalues`

In [21]:
# reset the helicopters' capacity to the original values
Helicopters = [15,25]*2

In [46]:
tab = texttable.Texttable()

headings = ['score', 'teams (labels)', 'assets (labels)']
tab.header(headings)

Ntasks = [4, 6, 8]
Nvalues = [3, 5, 7]

Hfunc = np.random.rand(len(Helicopters))

for nt in Ntasks:
    for nv in Nvalues:
        # generate demand
        demand = asset_demand(nassets, nt, nv, teamsize)
        # generate staff available
        staff = staff_available(nv)
        # form allocatable teams 
        teams, left = demand_staff_assign(demand, staff, nv)
        # allocate teams 
        Patterns = daily_teams_allocation(Helicopters, Hfunc, teams)
        # find highest value patterns
        maxV_patterns, SV = max_value_patterns(Patterns)
        # find smallest number of used helicopters patterns
        minH_patterns, SH = least_helicopters_patterns(maxV_patterns)
        # find largest allocated teams patterns
        maxN_patterns, SN = most_teams_patterns(minH_patterns)
        # find most efficient asset location disposition patterns
        maxA_patterns, SA = similar_destinations_patterns(maxN_patterns)
        # find highest flying probability patterns
        maxP_patterns, SP = max_probability_patterns(maxA_patterns)

        # extract data of optimal pattern
        H = maxP_patterns['H'][0]
        T = maxP_patterns['T'][0]
        A = maxP_patterns['A'][0]
        
        # prune empty lists corresponding to unused helicopters
        h = [i for (i,a) in enumerate(T) if a]
        H = [Helicopters[j] for j in h]
        T = [T[j] for j in h]
        A = [A[j] for j in h]

        score = SV * SH * SN * SA * SP
        tab.add_row([score, T, A])

tab.set_cols_align(['c','l','l'])
tab.set_cols_width([7,70,70])
s = tab.draw()
print(s)

+---------+------------------------------------------------------------------------+------------------------------------------------------------------------+
|  score  |                             teams (labels)                             |                            assets (labels)                             |
+=========+========================================================================+========================================================================+
|  0.655  | [['T_6', 'T_5', 'T_4'], ['T_3', 'T_2', 'T_1']]                         | [['A_9', 'A_5', 'A_7'], ['A_3', 'A_10', 'A_10']]                       |
+---------+------------------------------------------------------------------------+------------------------------------------------------------------------+
|  4.215  | [['T_8', 'T_7', 'T_6', 'T_5', 'T_4', 'T_3'], ['T_2', 'T_1']]           | [['A_9', 'A_1', 'A_8', 'A_2', 'A_10', 'A_7'], ['A_2', 'A_6']]          |
+---------+-----------------------------------------

### Differing selection criteria application

Let's test the robustness of the pattern selection functions defined above when applying them in a different (random) order.  

First, let's derive a reference pattern

In [47]:
# simulate daily assets' demand(s)
demand = asset_demand(nassets, ntasks, nvalues, teamsize)
# simulate daily available staff
staff = staff_available(nvalues)
# match demands with staff to create teams
Teams, left = demand_staff_assign(demand, staff, nvalues)
# simulate daily helicopters functionality
Hfunc = np.random.rand(len(Helicopters))
# build possible allocation patterns
Patterns = daily_teams_allocation(Helicopters, Hfunc, Teams)

# rename pattern selection function
selection1 = max_value_patterns
selection2 = least_helicopters_patterns
selection3 = most_teams_patterns
selection4 = similar_destinations_patterns
selection5 = max_probability_patterns

patterns = Patterns
patterns, SV = selection1(patterns)
patterns, SH = selection2(patterns)
patterns, SN = selection3(patterns)
patterns, SA = selection4(patterns)
patterns, SP = selection5(patterns)

AA = patterns['A'][0]
TT = patterns['T'][0]

h = [i for (i,a) in enumerate(AA) if a]
A = [AA[j] for j in h]
T = [TT[j] for j in h]

score = SV * SH * SN * SA * SP
score_ref, T_ref, A_ref = score, T, A

Using the same patterns list - resulting from the allocation phase with the function `demand_staff_assign` -  
we repeat the selection process multiple times, shuffling the order of the functions' application at each iteration.  
The result should not vary depending on these orderings

In [48]:
tab1 = texttable.Texttable()

headings = ['Score', 'teams (labels)', 'assets (labels)']
tab1.header(headings)

tab1.add_row([score_ref, T_ref, A_ref])

tab1.set_cols_align(['c','l','l'])
tab1.set_cols_width([5,80,80])
s = tab1.draw()
print('Reference')
print(s)
print()

tab2 = texttable.Texttable()

headings = ['score', 'teams (labels)', 'assets (labels)']
tab2.header(headings)

selections = np.arange(5)
scores = ['SV', 'SH', 'SN', 'SA', 'SP']

iterations = 10
patterns = Patterns
for it in range(iterations):

    for sel in selections:
        exec('patterns,{} = selection{}(patterns)'.format(scores[sel], sel+1))
    
    # extract data of optimal pattern
    AA = patterns['A'][0]
    TT = patterns['T'][0]
    
    h = [i for (i,a) in enumerate(AA) if a]
    A = [AA[j] for j in h]
    T = [TT[j] for j in h]
    
    score = SV * SH * SN * SA * SP
    tab2.add_row([score, T, A])
    
    shuffle(selections)

tab2.set_cols_align(['c','l','l'])
tab2.set_cols_width([5,80,80])
s = tab2.draw()
print('pattern selection shufflings')
print(s)

Reference
+-------+----------------------------------------------------------------------------------+----------------------------------------------------------------------------------+
| Score |                                  teams (labels)                                  |                                 assets (labels)                                  |
+=======+==================================================================================+==================================================================================+
| 0.195 | [['T_9', 'T_8', 'T_7', 'T_6', 'T_5', 'T_4', 'T_3'], ['T_2', 'T_1']]              | [['A_5', 'A_7', 'A_9', 'A_5', 'A_2', 'A_4', 'A_4'], ['A_6', 'A_8']]              |
+-------+----------------------------------------------------------------------------------+----------------------------------------------------------------------------------+

pattern selection shufflings
+-------+-----------------------------------------------------------------------

This shows that the result of the method is independent of the application order  
of the optimal pattern selection functions